In [ ]:
import os

def mkdir(nonspam_train):
    try:
        os.mkdir(nonspam_train)
        print('made', nonspam_train)
    except FileExistsError:
        print(nonspam_train, 'exists')

In [ ]:
mkdir('nonspam_train')

In [ ]:
mkdir('spam_train')

In [ ]:
mkdir('nonspam_test')

In [ ]:
mkdir('spam_test')

In [ ]:
import math
import re
# for sorting dictionaries
from collections import OrderedDict
from operator import itemgetter
from os import listdir
from os.path import isfile, join

from nltk.corpus import stopwords

In [ ]:
# number of features
m = 1000
spam_train_dir = "/content/spam_train/"
ham_train_dir = "/content/nonspam_train/"

feature_dictionary_dir = "/content/feature_dictionary.txt"


In [ ]:
# FUNCTIONS #

# defines the label of the files based on their names
def read_labels(files):
    labels = []
    for file in files:
        if "spam" in str(file):
            labels.append(1)
        elif "ham" in str(file):
            labels.append(0)
    return labels


def get_label_frequency(train_labels, class_label):
    frequency = 0
    for train_label in train_labels:
        if train_label == class_label:
            frequency = frequency + 1
    return frequency


def read_file(filename):
    text_file = open(filename, "r")
    text = text_file.read()
    return text


In [ ]:
# extracts tokens from the given text
def getTokens(text):
    text_tokens = re.findall(r"[\w']+", text)
    # remove digits, special characters and convert to lowercase
    for k in range(len(text_tokens)):
        text_tokens[k] = text_tokens[k].lower()
        text_tokens[k] = text_tokens[k].replace("_", "")
        text_tokens[k] = re.sub("[0-9]+", "", text_tokens[k])
    text_tokens = set(text_tokens)  # convert list to set, in order to remove duplicate tokens

    return text_tokens


def write_tokens_to_file(tokens, filename):
    f = open(filename, 'w')
    for token in tokens:
        f.write(token + '\n')
    f.close()

In [ ]:
# MAIN #

spam_train_files = sorted([f for f in listdir(spam_train_dir) if isfile(join(spam_train_dir, f))])
ham_train_files = sorted([f for f in listdir(ham_train_dir) if isfile(join(ham_train_dir, f))])

train_files = list(spam_train_files)
train_files.extend(ham_train_files)

train_labels = [1] * len(spam_train_files)
train_labels.extend([0] * len(ham_train_files))

stop_words = set(stopwords.words('english'))

no_of_train_files = len(train_files)

spam_class_frequency = len(spam_train_files)  # 1 is for SPAM, 0 is for HAM
print("number of SPAM train documents: " + str(spam_class_frequency))
ham_class_frequency = len(ham_train_files)  # 1 is for SPAM, 0 is for HAM
print("number of HAM train documents: " + str(ham_class_frequency))

spam_class_probability = spam_class_frequency / (len(spam_train_files) + len(ham_train_files))
print("SPAM train document probability: " + str(spam_class_probability))
ham_class_probability = ham_class_frequency / (len(spam_train_files) + len(ham_train_files))
print("HAM train document probability: " + str(ham_class_probability))

print('')

In [ ]:
# feature selection with Information Gain #

# a dictionary which has as a key how many documents a feature (token) appears in
feature_frequency = dict()

# a dictionary which has as a key how many train spam documents a feature appears in
feature_spam_frequency = dict()

# a dictionary which has as a key how many train ham documents a feature appears in
feature_ham_frequency = dict()

print('Calculating the frequency of each token...')

# calculate feature_frequencies dict
for i in range(len(train_files)):
    train_text = ''
    if train_labels[i] == 1:  # for "SPAM" files
        train_text = read_file(spam_train_dir + train_files[i])
    elif train_labels[i] == 0:  # for "HAM" files
        train_text = read_file(ham_train_dir + train_files[i])
    candidate_features = getTokens(train_text)

    for token in candidate_features:
        if token not in stop_words:
            if token not in feature_frequency:
                feature_frequency[token] = 1
            else:
                feature_frequency[token] = feature_frequency[token] + 1

            if train_labels[i] == 1:  # for "SPAM" files
                if token not in feature_spam_frequency:
                    feature_spam_frequency[token] = 1
                    feature_ham_frequency[token] = 0
                else:
                    feature_spam_frequency[token] = feature_spam_frequency[token] + 1
            elif train_labels[i] == 0:  # for "HAM" files
                if token not in feature_ham_frequency:
                    feature_ham_frequency[token] = 1
                    feature_spam_frequency[token] = 0
                else:
                    feature_ham_frequency[token] = feature_ham_frequency[token] + 1

# sort feature_frequency dictionary in descending order by frequency
feature_frequency = OrderedDict(sorted(feature_frequency.items(), key=itemgetter(1), reverse=True))

In [ ]:
# a dictionary which has as a key the probability of a feature appearing in a document
feature_probability = dict()

# a dictionary which has as a key the probability of a feature appearing in a train spam document
feature_spam_cond_probability = dict()

# a dictionary which has as a key the probability of a feature appearing in a train ham document
feature_ham_cond_probability = dict()

# a dictionary that contains the Information gain for each token
IG = dict()

# First calculate the entropy of the dataset
H_C = - (spam_class_probability * math.log(spam_class_probability) +
         ham_class_probability * math.log(ham_class_probability))

print('entropy of the dataset: H(C) = ' + str(H_C))

# precaution to avoid division by zero and log(0)
error = 1e-7

# Calculate the information gain for each candidate feature.
# The feature that decreases the entropy less is the most desired feature,
# because that means that it is capable of achieving better classification.
for (i, token) in enumerate(feature_frequency):
    if token != "":  # exclude the empty string ""
        feature_probability[token] = feature_frequency[token] / no_of_train_files  # P(Xi=1)
        feature_ham_cond_probability[token] = feature_ham_frequency[token] / ham_class_frequency  # P(Xi=1|C=0)
        feature_spam_cond_probability[token] = feature_spam_frequency[token] / spam_class_frequency  # P(Xi=1|C=1)

        # bayes rule: P(C=1|Xi=1) = P(Xi=1|C=1) * P(C=1) / P(Xi=1)
        P_C1_given_X1 = feature_spam_cond_probability[token] * spam_class_probability / \
                        (feature_probability[token] + error)
        # bayes rule: P(C=0|Xi=1) = P(Xi=1|C=0) * P(C=0) / P(Xi=1)
        P_C0_given_X1 = feature_ham_cond_probability[token] * ham_class_probability / \
                        (feature_probability[token] + error)

        # conditional entropy: H(C|Xi=1)
        H_C_given_X1 = - (P_C1_given_X1 * math.log(P_C1_given_X1 + error) +
                          P_C0_given_X1 * math.log(P_C0_given_X1 + error))

        # bayes rule: P(C=1|Xi=0) = P(Xi=0|C=1) * P(C=1) / P(Xi=0)
        P_C1_given_X0 = (1 - feature_spam_cond_probability[token]) * spam_class_probability / \
                        (1 - feature_probability[token] + error)
        # bayes rule: P(C=0|Xi=0) = P(Xi=0|C=0) * P(C=0) / P(Xi=0)
        P_C0_given_X0 = (1 - feature_ham_cond_probability[token]) * ham_class_probability / \
                        (1 - feature_probability[token] + error)

        # conditional entropy: H(C|Xi=0)
        H_C_given_X0 = - (P_C1_given_X0 * math.log(P_C1_given_X0 + error) +
                          P_C0_given_X0 * math.log(P_C0_given_X0 + error))

        # IG(C,Xi) = IG(Xi,C) = H(C) - SUM ( P(Xi=X_train) * H(C|Xi=X_train) for every X_train)
        IG[token] = H_C - (feature_probability[token] * H_C_given_X1 + (1 - feature_probability[token]) * H_C_given_X0)

        # print('{0}: P(Xi=1): {1}, P(Xi=1|C=0): {2}, P(Xi=1|C=1): {3}'.format(
        #     token,
        #     feature_probability[token],
        #     feature_ham_cond_probability[token],
        #     feature_spam_cond_probability[token]
        # ))

# MY ALTERNATIVE IG score calculation implementation
# Calculate the information gain for each candidate feature.
# IG is defined as the difference between the two conditional probabilities.
# The tokens where this difference is higher have higher Information Gain.
"""
feature_ham_probability = dict()
feature_spam_probability = dict()
for (i, token) in enumerate(feature_frequency):
    if token != "":  # exclude the empty string ""
        feature_probability[token] = feature_frequency[token] / no_of_train_files
        feature_ham_probability[token] = feature_ham_frequency[token] / ham_class_frequency
        feature_spam_probability[token] = feature_spam_frequency[token] / spam_class_frequency
        #IG[token] = feature_probability[token] * abs(feature_ham_probability[token] - feature_spam_probability[token])
        IG[token] = abs(feature_ham_probability[token] - feature_spam_probability[token])
        # print('{0}: P(Xi=1): {1}, P(Xi=1|C=0): {2}, P(Xi=1|C=1): {3}'.format(
        #     token,
        #     feature_probability[token],
        #     feature_ham_cond_probability[token],
        #     feature_spam_cond_probability[token]
        # ))
"""

# sort IG dictionary in descending order by score (the higher score the better)
IG = OrderedDict(sorted(IG.items(), key=itemgetter(1), reverse=True))

print('')

feature_tokens = []

# create and print the list of the feature dictionary tokens and their corresponding IG scores
print("feature tokens: ")
for (i, token) in enumerate(IG):
    # collect the m feature tokens with the highest information gain
    if i < m:
        feature_tokens.append(token)
        print(token + ", information gain score: " + str(IG[token]))
    else:
        break
print('')

# write feature_tokens to file
write_tokens_to_file(feature_tokens, feature_dictionary_dir)

In [ ]:
import math
import time
from os import listdir
from os.path import isfile, join
from nltk import word_tokenize
from nltk.corpus import stopwords

feature_dictionary_dir = "/content/feature_dictionary.txt"
spam_train_dir = "/content/spam_train/"
ham_train_dir = "/content/nonspam_train/"
spam_test_dir = "/content/spam_test/"
ham_test_dir = "/content/nonspam_test/"


In [ ]:
# FUNCTIONS #

def read_dictionary_file(filename):
    text_file = open(filename, "r")
    lines = text_file.readlines()
    for i in range(len(lines)):
        lines[i] = lines[i].replace("\n", "")
    return lines


def read_file(filename):
    text_file = open(filename, "r")
    text = text_file.read()
    return text


def calculate_token_frequencies_in_class(feature_tokens, stop_words, class_documents):
    token_frequencies_in_class = dict()  # same size as a feature vector
    class_distinct_words = set()
    total_words_in_class = 0

    for token in feature_tokens:
        token_frequencies_in_class[token] = 0

    # For each feature token count how many times the documents of the given class contain it.
    for i, document in enumerate(class_documents):
        # print('document:', document)
        tokenized_document = word_tokenize(document)
        # print('tokenized_document:', tokenized_document)
        filtered_document = [w.lower() for w in tokenized_document if not w.lower() in stop_words]
        # print('filtered_document:', filtered_document)
        for word in filtered_document:
            if word in feature_tokens:
                token_frequencies_in_class[word] = token_frequencies_in_class[word] + 1

        document_set = set(filtered_document)
        class_distinct_words = class_distinct_words.union(document_set)

        # number_of_class_words += len(tokenized_document)
        total_words_in_class += len(filtered_document)

    # number_of_class_words = len(class_distinct_words)
    return token_frequencies_in_class, class_distinct_words, total_words_in_class


def calculate_laplace_estimate_probability(
        test_feature_vector,
        feature_tokens,
        class_probability,
        token_frequencies_in_class,
        total_words_in_class,
        V
):
    # laplace_estimate_probability = 1
    laplace_estimate_log_probability = 0
    for i, test_feature in enumerate(test_feature_vector):
        token = feature_tokens[i]
        if test_feature == 1:
            if token in token_frequencies_in_class:
                probOfTokenBelongingToClass = (token_frequencies_in_class[token] + 1) / (total_words_in_class + V)
            else:
                probOfTokenBelongingToClass = (0 + 1) / (total_words_in_class + V)

            # traditional way: using multiplications of probabilities
            # laplace_estimate_probability *= probOfTokenBelongingToClass

            # numerically stable way to avoid multiplications of probabilities
            laplace_estimate_log_probability += math.log(probOfTokenBelongingToClass, 2)

    # laplace_estimate_probability *= class_probability
    laplace_estimate_log_probability += math.log(class_probability, 2)

    # return laplace_estimate_probability
    return laplace_estimate_log_probability


In [ ]:
# MAIN #

if __name__ == '__main__':

    start_time = time.time()

    spam_train_files = sorted([f for f in listdir(spam_train_dir) if isfile(join(spam_train_dir, f))])
    ham_train_files = sorted([f for f in listdir(ham_train_dir) if isfile(join(ham_train_dir, f))])
    spam_test_files = sorted([f for f in listdir(spam_test_dir) if isfile(join(spam_test_dir, f))])
    ham_test_files = sorted([f for f in listdir(ham_test_dir) if isfile(join(ham_test_dir, f))])

    train_files = list(spam_train_files)
    train_files.extend(ham_train_files)

    test_files = list(spam_test_files)
    test_files.extend(ham_test_files)

    train_labels = [1] * len(spam_train_files)
    train_labels.extend([0] * len(ham_train_files))

    test_true_labels = [1] * len(spam_test_files)
    test_true_labels.extend([0] * len(ham_test_files))

    spam_class_frequency = len(spam_train_files)  # 1 is for SPAM, 0 is for HAM
    print("number of SPAM train documents: " + str(spam_class_frequency))
    ham_class_frequency = len(ham_train_files)  # 1 is for SPAM, 0 is for HAM
    print("number of HAM train documents: " + str(ham_class_frequency))

    spam_class_probability = spam_class_frequency / (len(spam_train_files) + len(ham_train_files))
    print("SPAM train document probability: " + str(spam_class_probability))
    ham_class_probability = ham_class_frequency / (len(spam_train_files) + len(ham_train_files))
    print("HAM train document probability: " + str(ham_class_probability))

    print('')

In [ ]:
feature_tokens = read_dictionary_file(feature_dictionary_dir)
stop_words = set(stopwords.words('english'))


In [ ]:
print("Reading TRAIN files...")
spam_train_documents = []
ham_train_documents = []
for i in range(len(train_files)):
    if train_labels[i] == 1:  # for "SPAM" files
        spam_train_document = read_file(spam_train_dir + train_files[i])
        # candidate_features = getTokens(train_text)
        spam_train_documents.append(spam_train_document)
    elif train_labels[i] == 0:  # for "HAM" files
        ham_train_document = read_file(ham_train_dir + train_files[i])
        # candidate_features = getTokens(train_text)
        ham_train_documents.append(ham_train_document)
print('DONE\n')


In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
print("Calculating feature token frequencies in SPAM files...")
token_frequencies_in_spam_class, spam_distinct_words, total_words_in_spam_class = \
    calculate_token_frequencies_in_class(feature_tokens, stop_words, spam_train_documents)
print('DONE\n')

print("Calculating feature token frequencies in HAM files...")
token_frequencies_in_ham_class, ham_distinct_words, total_words_in_ham_class = \
    calculate_token_frequencies_in_class(feature_tokens, stop_words, ham_train_documents)
print('DONE\n')


In [ ]:
V = len(spam_distinct_words.union(ham_distinct_words))

print('total words in spam class:', total_words_in_spam_class)
print('total words in ham class:', total_words_in_ham_class)
print('vocabulary size |V|:', V)
print('')

wrong_counter = 0  # the number of wrong classifications made by Logistic Regression

true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0


In [ ]:
# testing files with Naive Bayes classifier using Laplace estimates
print("Reading TEST files...")
for i in range(len(test_files)):  # for all the test files that exist
    test_text = ''
    if test_true_labels[i] == 1:  # 1 is for class "SPAM"
        test_text = read_file(spam_test_dir + test_files[i])
    if test_true_labels[i] == 0:  # 0 is for class "HAM"
        test_text = read_file(ham_test_dir + test_files[i])
    test_text_tokens = word_tokenize(test_text)
    filtered_test_text_tokens = [w.lower() for w in test_text_tokens if not w.lower() in stop_words]

    test_feature_vector = [0] * len(feature_tokens)
    for j in range(len(feature_tokens)):
        if feature_tokens[j] in test_text_tokens:
            test_feature_vector[j] = 1


In [ ]:
spam_laplace_estimate_probability = calculate_laplace_estimate_probability(
    test_feature_vector,
    feature_tokens,
    spam_class_probability,
    token_frequencies_in_spam_class,
    total_words_in_spam_class,
    V
)


In [ ]:
 ham_laplace_estimate_probability = calculate_laplace_estimate_probability(
            test_feature_vector,
            feature_tokens,
            ham_class_probability,
            token_frequencies_in_ham_class,
            total_words_in_ham_class,
            V
        )

In [ ]:
if spam_laplace_estimate_probability >= ham_laplace_estimate_probability and test_true_labels[i] == 1:
    print("'" + test_files[i] + "'" + " classified as: SPAM -> correct")
    true_positives += 1
elif spam_laplace_estimate_probability >= ham_laplace_estimate_probability and test_true_labels[i] == 0:
    print("'" + test_files[i] + "'" + " classified as: SPAM -> WRONG!")
    wrong_counter += 1
    false_positives += 1
elif spam_laplace_estimate_probability < ham_laplace_estimate_probability and test_true_labels[i] == 0:
    print("'" + test_files[i] + "'" + " classified as: HAM -> correct")
    true_negatives += 1
elif spam_laplace_estimate_probability < ham_laplace_estimate_probability and test_true_labels[i] == 1:
    print("'" + test_files[i] + "'" + " classified as: HAM -> WRONG!")
    wrong_counter += 1
    false_negatives += 1

print('')


In [ ]:
print('Manual Naive-Bayes Classifier:')
print('Number of features used: ' + str(len(feature_tokens)))
print('')

# Accuracy
accuracy = ((len(test_files) - wrong_counter) / len(test_files)) * 100
print("Accuracy: " + str(accuracy) + " %")
print('')


# Total duration
print("Total duration: %s seconds" % (time.time() - start_time))
